##### Load Package

In [12]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator

import os
from dotenv import load_dotenv

##### Set Up Environment 

In [7]:
load_dotenv(dotenv_path = "../Key/.env")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# No Output Parser

In [8]:
chat = ChatOpenAI()

In [9]:
prompt_template = ChatPromptTemplate.from_template(
    '''
    You are a master brading consulatant who specializes in naming brands.
    You come up with catchy and memorable brand names. 
    Take the brand description below delimited by triple backticks and use it to create the name for a brand. 
    brand description: ```{brand_description}```
    based on the description give the score of 1-10 for how likely it is to success and provide the reason. 
    '''
)

prompt = prompt_template.format_messages(brand_description = "a sneaker that make running faster and comfortable.")

print(chat.invoke(prompt).content)

Brand Name: SpeedStride

Likelihood of Success: 9/10

Reasoning: The name "SpeedStride" effectively conveys the idea of running faster while also emphasizing comfort. It is catchy, memorable, and clearly communicates the main benefit of the product. The name is likely to resonate with customers looking for high-performance sneakers for running.


# StructuredOutputParser

In [10]:
response_schema = [
    ResponseSchema(name = "brand_name",
                   description="This is the name of the brand."),
    ResponseSchema(name = "likelihood_of_success",
                   description="This is an integer score between 1-10."),
    ResponseSchema(name = "reasoning",
                   description="This is the reasons for the score.")]

output_parser = StructuredOutputParser.from_response_schemas(response_schema)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"brand_name": string  // This is the name of the brand.
	"likelihood_of_success": string  // This is an integer score between 1-10.
	"reasoning": string  // This is the reasons for the score.
}
```


In [13]:
prompt_template = ChatPromptTemplate.from_template(
    '''
    You are a master brading consulatant who specializes in naming brands.
    You come up with catchy and memorable brand names. 
    Take the brand description below delimited by triple backticks and use it to create the name for a brand. 
    brand description: ```{brand_description}```
    based on the description give the score of 1-10 for how likely it is to success and provide the reason. 

    {format_instructions} 
    '''
)

prompt = prompt_template.format_messages(brand_description = "a sneaker that make running faster and comfortable.",
                                         format_instructions = format_instructions)

print(prompt)

print(chat.invoke(prompt).content)

[HumanMessage(content='\n    You are a master brading consulatant who specializes in naming brands.\n    You come up with catchy and memorable brand names. \n    Take the brand description below delimited by triple backticks and use it to create the name for a brand. \n    brand description: ```a sneaker that make running faster and comfortable.```\n    based on the description give the score of 1-10 for how likely it is to success and provide the reason. \n\n    The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"brand_name": string  // This is the name of the brand.\n\t"likelihood_of_success": string  // This is an integer score between 1-10.\n\t"reasoning": string  // This is the reasons for the score.\n}\n``` \n    ')]
```json
{
	"brand_name": "SpeedStride",
	"likelihood_of_success": "8",
	"reasoning": "The name suggests speed and comfort, which are key features of the sneaker. It 

In [14]:
prompt_template = PromptTemplate(
    template = '''
    You are a master brading consulatant who specializes in naming brands.
    You come up with catchy and memorable brand names. 
    Take the brand description below delimited by triple backticks and use it to create the name for a brand. 
    brand description: ```{brand_description}```
    based on the description give the score of 1-10 for how likely it is to success and provide the reason. 

    {format_instructions} 
    ''',
    input_variables= ["brand_description"],
    partial_variables= {"format_instructions": format_instructions}
)


prompt = prompt_template.format(brand_description = "a sneaker that make running faster and comfortable.")

print(prompt)

print(chat.invoke(prompt).content)


    You are a master brading consulatant who specializes in naming brands.
    You come up with catchy and memorable brand names. 
    Take the brand description below delimited by triple backticks and use it to create the name for a brand. 
    brand description: ```a sneaker that make running faster and comfortable.```
    based on the description give the score of 1-10 for how likely it is to success and provide the reason. 

    The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"brand_name": string  // This is the name of the brand.
	"likelihood_of_success": string  // This is an integer score between 1-10.
	"reasoning": string  // This is the reasons for the score.
}
``` 
    
```json
{
	"brand_name": "SpeedStride",
	"likelihood_of_success": "8",
	"reasoning": "The name suggests both speed and comfort, which are key factors for a successful running sneaker brand. It is catchy and mem

# Pydantic Output Parser

In [20]:
class BrandInfo(BaseModel):
    brand_name: str = Field(description = "This is the name of the brand.")
    reasoning: str = Field(description = "This is the reasons for the score.")
    likelihood_of_success: int = Field(description = "This is an integer score between 1-10.")

    @validator("likelihood_of_success")
    def check_score(cls, field):
        if field > 10:
            raise ValueError("Wrong Score")
        return field

pydantic_parser = PydanticOutputParser(pydantic_object = BrandInfo)

format_instructions = pydantic_parser.get_format_instructions()

In [21]:
prompt_template = ChatPromptTemplate.from_template(
    '''
    You are a master brading consulatant who specializes in naming brands.\
    You come up with catchy and memorable brand names. \
    Take the brand description below delimited by triple backticks and use it to create the name for a brand. \
    brand description: ```{brand_description}```
    based on the description give the score of 1-10 for how likely it is to success and provide the reason. 

    {format_instructions} 
    '''
)

prompt = prompt_template.format_messages(brand_description = "a sneaker that make running faster and comfortable.",
                                         format_instructions = format_instructions)

print(chat.invoke(prompt).content)


{
    "brand_name": "SpeedSole",
    "reasoning": "The name suggests speed and comfort, which aligns well with the description of the sneaker.",
    "likelihood_of_success": 8
}
